# Project 2 team 23

### data and method preparation

In [35]:
import numpy as np 
import pandas as pd
import gurobipy as gp
from gurobipy import *


airports = pd.read_csv("airports.csv")
airports = airports.drop(airports.columns[0], axis=1)
airports.head(5)
print(len(airports))

from haversine import haversine, Unit

def distance(colnum1, colnum2):
    # input: 2 col number of airports dataframe
    lat1= airports.loc[colnum1,'latitude_deg']
    lon1 = airports.loc[colnum1,'longitude_deg']
    lat2 = airports.loc[colnum2,'latitude_deg']
    lon2 = airports.loc[colnum2,'longitude_deg']
    return haversine((lat1, lon1), (lat2, lon2), unit=Unit.MILES)

warehouses = [0,1] # from dataframe, JFK has col number 0, LAX has 1


45


## Problem 1

In [36]:
m = gp.Model("problem1")

# adding variables: 1 - edges 
e = {}

for i in range(len(airports)):
    for j in range(len(airports)):
        if i != j:
            e[i, j] = m.addVar(vtype = GRB.INTEGER, lb=0, name = f"e_{i}_{j}")

# adding variables: 2 - subtour eliminations
u = {}
for i in range(len(airports)):
    u[i] = m.addVar(vtype=GRB.CONTINUOUS, lb=1, ub=len(airports), name = f"u_{i}")

# adding objective function: total distance
m.setObjective(quicksum(e[i, j] * distance(i,j) 
                        for i in range(len(airports)) 
                        for j in range(len(airports)) 
                        if i != j),GRB.MINIMIZE)

In [37]:
# adding constraint: 1 - in and out have the same value for each airport
for i in range(2,len(airports)):
            m.addConstr(quicksum(e[i,j] for j in range(len(airports)) if i!=j) == 
                        quicksum(e[j,i] for j in range(len(airports)) if i!=j)) 

# adding constraint: 2 - each airport (i) are left and entered exactly once (except for JKF & LAX, there might not be a route)
for i in range(2, len(airports)):
    m.addConstr(quicksum(e[i,j] for j in range(len(airports)) if j!=i)==1)
    m.addConstr(quicksum(e[j,i] for j in range(len(airports)) if j!=i)==1)

# adding constraint: 3 - there should be one plane starting JKF, one ending at LAX
m.addConstr(quicksum(e[0,j] for j in range(2,len(airports)))==1)
m.addConstr(quicksum(e[j,1] for j in range(2,len(airports)))==1)


# adding constraint: 4 - there's no travel between JKF/LAX
m.addConstr(e[0,1] == 0)
m.addConstr(e[1,0] == 0)

# starts from JFK, ends in LAX
m.addConstr(u[0] == 1)  
m.addConstr(u[1] == len(airports))  

# Subtour elimination constraints (MTZ)

for i in range(2, len(airports)):
    for j in range(2, len(airports)):
        if i != j:
            m.addConstr(u[i] +1 <= u[j] + (len(airports)-1)* (1 - e[i, j]))

# Set parameters for near-optimal solution
m.setParam("MIPGap", 0.1)

# Solve the model
m.optimize()

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 1941 rows, 2025 columns and 13076 nonzeros
Model fingerprint: 0xb662d1cb
Variable types: 45 continuous, 1980 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [6e+01, 1e+04]
  Bounds range     [1e+00, 4e+01]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 251072.25799
Presolve removed 47 rows and 4 columns
Presolve time: 0.02s
Presolved: 1894 rows, 2021 columns, 9288 nonzeros
Variable types: 43 continuous, 1978 integer (1978 binary)

Root relaxation: objective 2.570066e+04, 173 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [38]:
import folium

edges = []
for key in e.keys():
    if e[key].X > 0:
        if len(key) >= 2:
            i, j = key[-2], key[-1]
            edges.append((i, j))

print("Selected edges:", edges)

map_problem1 = folium.Map(location=[20, 0], zoom_start=2)

for index, row in airports.iterrows():
    folium.Marker([row['latitude_deg'], row['longitude_deg']], popup=row['airport_code']).add_to(map_problem1)

for i, j in edges:
    coord_i = [airports.loc[i, 'latitude_deg'], airports.loc[i, 'longitude_deg']]
    coord_j = [airports.loc[j, 'latitude_deg'], airports.loc[j, 'longitude_deg']]
    #print(f"Drawing line from {coord_i} to {coord_j}")
    folium.PolyLine([coord_i, coord_j], weight=2.5, opacity=1).add_to(map_problem1)

map_problem1

Selected edges: [(0, 43), (2, 41), (3, 23), (4, 21), (5, 9), (6, 33), (7, 31), (8, 44), (9, 15), (10, 30), (11, 14), (12, 32), (13, 26), (14, 13), (15, 24), (16, 6), (17, 3), (18, 7), (19, 17), (20, 12), (21, 38), (22, 11), (23, 39), (24, 22), (25, 2), (26, 34), (27, 19), (28, 42), (29, 4), (30, 18), (31, 16), (32, 8), (33, 5), (34, 29), (35, 10), (36, 40), (37, 1), (38, 25), (39, 35), (40, 28), (41, 36), (42, 20), (43, 27), (44, 37)]


## Problem 2
### Setting up the model: 1 - variables and objectives

### Setting up the model: 2.1 - Case 1 Variables, Objectives Constraints and model optimization

In [39]:
m = gp.Model("problem2-case1")

# adding variables: 1 - edges 
e = {}
for start in warehouses:
    for i in range(len(airports)):
        for j in range(len(airports)):
            if i != j:
                e[start, i, j] = m.addVar(vtype = GRB.INTEGER, name = f"e_{start}_{i}_{j}")

# adding variables: 2 - subtour eliminations
u = {}
for start in warehouses:
    for i in range(len(airports)):
        u[start, i] = m.addVar(vtype=GRB.CONTINUOUS, lb=1, ub=len(airports), name = f"u_{start}_{i}")

# adding objective function: total distance
m.setObjective(quicksum(e[start, i, j] * distance(i,j) 
                        for start in warehouses for i in range(len(airports)) for j in range(len(airports)) if i != j),GRB.MINIMIZE)

# adding constraint: 1 - in and out have the same value for each airport
for start in warehouses:
    for i in range(len(airports)):
                m.addConstr(quicksum(e[start,i,j] for j in range(len(airports)) if i!=j) == quicksum(e[start,j,i] for j in range(len(airports)) if i!=j)) 

# adding constraint: 2 - each airport (i) are left and entered exactly once (except for JKF & LAX, there might not be a route)
for i in range(2,len(airports)):
    m.addConstr(quicksum(e[start,i,j] for start in warehouses for j in range(len(airports)) if j!=i)==1)
    m.addConstr(quicksum(e[start,j,i] for start in warehouses for j in range(len(airports)) if j!=i)==1)

# adding constraint: 3 - there should be *2 planes* plane starting from JKF/LAX
m.addConstr(quicksum(e[0, 0, i]+e[1, 1, i] for i in range(2, len(airports))) == 2)

# adding constraint: 4 - if any plane leaves from JKF/LAX, it comes back...
for start in warehouses:
    m.addConstr(quicksum(e[start,start,i] for i in range(2,len(airports))) == quicksum(e[start,j,start] for j in range(2,len(airports))))


# adding constraint: 5 - there's no travel between starting airports
m.addConstr(e[0,0,1] == 0)
m.addConstr(e[1,1,0] == 0)

# Position of the first node - u[0] = 1
for k in warehouses:
    m.addConstr(u[k, 0] == 1)  

# Subtour elimination constraints (MTZ)
for k in warehouses:
    for i in range(2, len(airports)):
        for j in range(2, len(airports)):
            if i != j:
                m.addConstr(u[k, i] +1 <= u[k, j] + (len(airports)-1)* (1 - e[k, i, j]))

# Set parameters for near-optimal solution
m.setParam("MIPGap", 0.1)

# Solve the model
m.optimize()






Set parameter MIPGap to value 0.1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3795 rows, 4050 columns and 26586 nonzeros
Model fingerprint: 0xe9738467
Variable types: 90 continuous, 3960 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [6e+01, 1e+04]
  Bounds range     [1e+00, 4e+01]
  RHS range        [1e+00, 4e+01]
Presolve removed 6 rows and 8 columns
Presolve time: 0.02s
Presolved: 3789 rows, 4042 columns, 26402 nonzeros
Variable types: 86 continuous, 3956 integer (3956 binary)
Found heuristic solution: objective 502593.39901
Found heuristic solution: objective 167948.61334

Root relaxation: objective 2.754063e+04, 127 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | I

### Setting up the model: 2.2 - Case 2 Variables, Objectives Constraints and model optimization

In [40]:
m = gp.Model("problem2-case2")

# adding variables: 1 - edges 
e = {}
for start in warehouses:
    for i in range(len(airports)):
        for j in range(len(airports)):
            if i != j:
                e[start, i, j] = m.addVar(vtype = GRB.INTEGER, name = f"e_{start}_{i}_{j}")

# adding variables: 2 - subtour eliminations
u = {}
for start in warehouses:
    for i in range(len(airports)):
        u[start, i] = m.addVar(vtype=GRB.CONTINUOUS, lb=1, ub=len(airports), name = f"u_{start}_{i}")

# adding objective function: total distance
m.setObjective(quicksum(e[start, i, j] * distance(i,j) 
                        for start in warehouses for i in range(len(airports)) for j in range(len(airports)) if i != j),GRB.MINIMIZE)

# adding constraint: 1 - in and out have the same value for each airport
for start in warehouses:
    for i in range(len(airports)):
                m.addConstr(quicksum(e[start,i,j] for j in range(len(airports)) if i!=j) == quicksum(e[start,j,i] for j in range(len(airports)) if i!=j)) 

# adding constraint: 2 - each airport (i) are left and entered exactly once (except for JKF & LAX, there might not be a route)
for i in range(2,len(airports)):
    m.addConstr(quicksum(e[start,i,j] for start in warehouses for j in range(len(airports)) if j!=i)==1)
    m.addConstr(quicksum(e[start,j,i] for start in warehouses for j in range(len(airports)) if j!=i)==1)

# adding constraint: 3 - there should be at least one plane starting from JKF/LAX
m.addConstr(quicksum(e[0, 0, i]+e[1, 1, i] for i in range(2, len(airports))) >= 1)

# adding constraint: 4 - if any plane leaves from JKF/LAX, it comes back...
for start in warehouses:
    m.addConstr(quicksum(e[start,start,i] for i in range(2,len(airports))) == quicksum(e[start,j,start] for j in range(2,len(airports))))


# adding constraint: 5 - there's no travel between starting airports
m.addConstr(e[0,0,1] == 0)
m.addConstr(e[1,1,0] == 0)

# Position of the first node - u[0] = 1
for k in warehouses:
    m.addConstr(u[k, 0] == 1)  

# Subtour elimination constraints (MTZ)
for k in warehouses:
    for i in range(2, len(airports)):
        for j in range(2, len(airports)):
            if i != j:
                m.addConstr(u[k, i] +1 <= u[k, j] + (len(airports)-1)* (1 - e[k, i, j]))

# Set parameters for near-optimal solution
m.setParam("MIPGap", 0.1)

# Solve the model
m.optimize()






Set parameter MIPGap to value 0.1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3795 rows, 4050 columns and 26586 nonzeros
Model fingerprint: 0x10b8e870
Variable types: 90 continuous, 3960 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [6e+01, 1e+04]
  Bounds range     [1e+00, 4e+01]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 236955.65688
Presolve removed 6 rows and 8 columns
Presolve time: 0.03s
Presolved: 3789 rows, 4042 columns, 26402 nonzeros
Variable types: 86 continuous, 3956 integer (3956 binary)

Root relaxation: objective 2.460151e+04, 126 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [41]:
import folium

# Create a dictionary of coordinates
edges = [(k , i, j) for k in [0,1] for i in range(0, 45) for j in range(0, 45) if i != j and e[k ,i, j].X > 0.5]
map = folium.Map(location=[20, 0], zoom_start=2)
for index, row in airports.iterrows():
    folium.Marker([row['latitude_deg'],row['longitude_deg']], popup=row['airport_code']).add_to(map)
for x in edges:
    coord_i = [airports.loc[x[1],'latitude_deg'], airports.loc[x[1],'longitude_deg']]
    coord_j = [airports.loc[x[2],'latitude_deg'], airports.loc[x[2],'longitude_deg']]
    folium.PolyLine([coord_i,coord_j], weight=2.5, opacity=1).add_to(map)

map